# First Time Setup Processes

In [ ]:
import numpy as np
import pandas as pd

import bokeh
import hvplot.pandas
import holoviews as hv

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()

In [ ]:
#create the client:
   #first log into Neuprint online and copy auth token in account info. Insert auth token below
   #insert hemibrain dataset version into fetch_version()
   #must create new client if restart/clear outputs
    
#to access Neuprint data externally, go to the https://neuprint.janelia.org/?dataset=hemibrain:v1.2.1&qt=findneurons
    #in upper right corner, click on the “my account” icon → account → Auth Token
    #copy and paste the authentication token into the API to use externally

from neuprint import Client

c = Client('neuprint.janelia.org', dataset='hemibrain:v1.2.1', token='')
c.fetch_version()

In [ ]:
#test for successful client initialization.
  #should say "Found 177 results"

q = """\
    MATCH (n :Neuron {`AB(R)`: true})
    WHERE n.pre > 10
    RETURN n.bodyId AS bodyId, n.name AS name, n.pre AS numpre, n.post AS numpost
    ORDER BY n.pre + n.post DESC
"""

results = c.fetch_custom(q)

print(f"Found {len(results)} results")

# Search Criteria

# Functions

In [ ]:
#fetch neuron properties by body ID or neuron type
#to switch desired search criteria, assign value to preferred variable & insert variable into fetch_neurons()

list = [880875742, 910783805910787908, 940787029, 941814244, 941831818, 942959557, 943269573, 973894912, 973963898, 1003672833, 1035640810, 1066748608, 1095271087,1095279301, 1097407853, 1125969140, 1128097470, 1128757926, 1188388754, 5813016150, 5813041382, 5813069339, 5813087532]
criteria = 'PEN_b(PEN2)'

from neuprint import fetch_neurons, NeuronCriteria as NC

neuron_df, roi_counts_df = fetch_neurons(list) #change variable here to change query type

neuron_df[['bodyId', 'instance', 'type', 'pre', 'post', 'status', 'cropped', 'size']]

In [ ]:
#neuprint's program

#fetch neuron connections: all direct connections between upstream/downstream neurons. 
#independent from above, but required for the below table/matrix.
#can query by various parameters a manual list or by type as specified in the current code

list = [1344258473, 1375366517]

from neuprint import fetch_adjacencies, NeuronCriteria as NC
neuron_df, conn_df = fetch_adjacencies(NC(type='IbSpsP.*', regex=True), NC(type='Delta.*', regex=True), min_roi_weight=0, min_total_weight=0)

conn_df.sort_values('weight')

## Generate a pandas dataframe connectivity matrix and convert to exported .csv file

In [ ]:
#Numpy Array Connectivity Matrix of neurons queried by type. No current ability to order neurons by instance or subregional
    #localization; if fine ordering desired, must manually generate list

from neuprint import fetch_neurons, fetch_adjacencies, NeuronCriteria as NC

pre_criteria = 'IbSpsP'
pre_df, roi_counts_df = fetch_neurons(pre_criteria)
list_post = trial_df['bodyId'].tolist()

post_criteria = 'Delta'
post_df, roi_counts_df = fetch_neurons(post_criteria)
list_pre = trial_df['bodyId'].tolist()

weights = np.zeros(((len(list_pre)+1), (len(list_post))+1), dtype = np.uint64)

for i in range(len(list_pre)):
    for j in range(len(list_post)):
        if j == 0:
            weights[i+1,j] = list_pre[i]
                
        if i == 0:
            weights[i,j+1] = list_post[j]
        
        
        try:
            neuron_df, conn_df = fetch_adjacencies(list_pre[i], list_post[j])
        except KeyError:
            print(i)
            print(j)
            print("")
            weight = 9999
        else:
            if conn_df.empty:
                weight = 0
            else:
                weight = int(conn_df["weight"].sum())

        
        weights[i+1,j+1] = weight
        
sample_df = pd.DataFrame(weights)
sample_df.to_csv('IbSpsP_to_GnL0_array_ordered.csv')        
        
#print(weights)
print('Conversion Complete')

In [ ]:
#Numpy Array Connectivity Matrix of manually generated list of pre and post neurons. 
    #Pre-neurons on y-axis, post-neurons in x-axis. Exports to :C\Users\

#this version enables manual ordering of neurons (desirable if for ex. ordering neurons by glomerular localization on axes)

from neuprint import fetch_adjacencies, NeuronCriteria as NC

#GnL0
list_post = [1375366517, 1344258473]
#IbSpsP
list_pre = [5813069339, 1066748608, 1003672833, 942959557, 880875742, 5813041382, 973963898, 1128097470, 943269573, 1097407853, 1128757926, 1035640810, 5813016150, 910787908, 941831818, 973894912, 910783805, 940787029, 941814244, 1095271087, 1125969140, 5813087532, 1188388754, 1095279301]         

weights = np.zeros(((len(list_pre)+1), (len(list_post))+1), dtype = np.uint64)

for i in range(len(list_pre)):
    for j in range(len(list_post)):
        if j == 0:
            weights[i+1,j] = list_pre[i]
                
        if i == 0:
            weights[i,j+1] = list_post[j]
        
        
        try:
            neuron_df, conn_df = fetch_adjacencies(list_pre[i], list_post[j])
        except KeyError:
            print(i)
            print(j)
            print("")
            weight = 9999
        else:
            if conn_df.empty:
                weight = 0
            else:
                weight = int(conn_df["weight"].sum())

        
        weights[i+1,j+1] = weight
        
sample_df = pd.DataFrame(weights)
sample_df.to_csv('IbSpsP_to_GnL0_array_ordered.csv')        
        
#print(weights)
print('Conversion Complete')
        
        


## Incomplete Code: Create Connectivity Matrix in format convertable to Heatmap

In [ ]:
#merge arbitrary neuron properties onto the connection table

from neuprint import merge_neuron_properties

print(conn_df)
conn_df = merge_neuron_properties(neuron_df, conn_df, ['type', 'instance'])



In [ ]:
#create connectivity matrix from above fetch neuron connections 
#must adjust matrix.iloc values to >/= number of pre/post neurons being queried

from neuprint.utils import connection_table_to_matrix

matrix = connection_table_to_matrix(conn_df, 'bodyId', sort_by='instance')
matrix.iloc[:50, :50]

#X-axis = post, Y-axis = pre

In [ ]:
#export above connectivity matrix as excel file
#assigned file_name must end in .xlsx
#dafault exports to C:Users/Name

conn_matrix = pd.DataFrame(matrix)
file_name = 'Test_ISP_Delta_order_matrix.xlsx'

conn_matrix.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

## Incomplete Code for Connectivity Matrix to Heat Map

In [ ]:
#create heat map from connectivity matrix

matrix.hvplot.heatmap(height=600, width=700).opts(xrotation=60)

In [ ]:
list = [1344258473, 1375366517]

from neuprint import fetch_adjacencies, NeuronCriteria as NC
neuron_df, conn_df = fetch_adjacencies(((list), Regex=True), NC(type='PEN_b.*', regex=True))

conn_df.sort_values('weight', ascending=False)